In [32]:
import pandas as pd 
import numpy as np 

In [34]:
df_target = pd.read_csv("Target datasets.csv", encoding='ISO-8859-1')
df_county_population_2023 = pd.read_csv("county_population_2023.csv", skiprows=1, dtype=str)
df_median_income_by_county_2023 = pd.read_csv("median_income_by_county_2023.csv", skiprows=1, dtype=str)


In [35]:
df_target.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780 entries, 0 to 1779
Data columns (total 47 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   X.locale                             1780 non-null   object 
 1   Address.AddressLine1                 1780 non-null   object 
 2   Address.AddressLine2                 56 non-null     object 
 3   Address.City                         1780 non-null   object 
 4   Address.CountryName                  1780 non-null   object 
 5   Address.County                       1779 non-null   object 
 6   Address.FormattedAddress             1780 non-null   object 
 7   Address.IntersectionDescription      1779 non-null   object 
 8   Address.Latitude                     1780 non-null   float64
 9   Address.Longitude                    1780 non-null   float64
 10  Address.PostalCode                   1780 non-null   object 
 11  Address.Subdivision           

In [36]:
df_county_population_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3224 entries, 0 to 3223
Columns: 379 entries, Geography to Unnamed: 378
dtypes: object(379)
memory usage: 9.3+ MB


In [37]:
df_median_income_by_county_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3224 entries, 0 to 3223
Columns: 243 entries, Geography to Unnamed: 242
dtypes: object(243)
memory usage: 6.0+ MB


In [38]:
df_target.describe()

,Address.Latitude,Address.Longitude,ID,Store.StoreDistrictID,Store.StoreGroupID,Store.StoreRegionID,TimeZone.TimeZoneOffset.OffsetHours
count,1780.000000,1780.000000,1780.000000,1780.000000,1780.000000,1780.000000,1780.000000
mean,37.726457,-93.826145,1569.673034,279.156180,333.823034,239.438202,-6.080337
std,5.232686,16.848969,780.810419,115.942264,113.077595,112.689633,1.170538
min,19.648413,-158.092822,3.000000,101.000000,190.000000,100.000000,-10.000000
25%,33.913814,-106.894099,988.750000,170.750000,197.000000,100.000000,-7.000000
50%,38.776776,-88.928629,1481.500000,264.000000,298.000000,200.000000,-6.000000
75%,41.626572,-80.318192,2194.250000,372.250000,395.250000,300.000000,-5.000000
max,61.578202,-68.741560,3295.000000,470.000000,499.000000,400.000000,-5.000000


In [39]:
df_county_population_2023

,Geography,Geographic Area Name,Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Total population!!Male,Margin of Error!!SEX AND AGE!!Total population!!Male,Estimate!!SEX AND AGE!!Total population!!Female,Margin of Error!!SEX AND AGE!!Total population!!Female,Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),Margin of Error!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females),...,"Percent Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Two or More Races!!Two races excluding Some Other Race, and three or more races",Percent!!Total housing units,Percent Margin of Error!!Total housing units,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female",Unnamed: 378
0,0100000US,United States,332387540,*****,164545087,6966,167842453,7054,98.0,0.1,...,0.1,(X),(X),238741214,(X),48.8,0.1,51.2,0.1,NaN
1,010C243US,United States -- Rural,66686736,51752,33961441,30653,32725295,32074,103.8,0.1,...,0.1,(X),(X),50921163,(X),50.7,0.1,49.3,0.1,NaN
2,0500000US01001,"Autauga County, Alabama",59285,*****,28669,263,30616,263,93.6,1.7,...,0.7,(X),(X),44761,(X),47.8,0.4,52.2,0.4,NaN
3,0500000US01003,"Baldwin County, Alabama",239945,*****,117316,153,122629,153,95.7,0.2,...,0.3,(X),(X),184665,(X),48.1,0.1,51.9,0.1,NaN
4,0500000US01005,"Barbour County, Alabama",24757,*****,12906,119,11851,119,108.9,2.1,...,0.4,(X),(X),19201,(X),53.3,0.3,46.7,0.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3219,0500000US72145,"Vega Baja Municipio, Puerto Rico",54058,*****,25765,*****,28293,*****,91.1,*****,...,0.1,(X),(X),44779,(X),46.9,0.1,53.1,0.1,NaN
3220,0500000US72147,"Vieques Municipio, Puerto Rico",8147,*****,4178,177,3969,177,105.3,9.1,...,0.6,(X),(X),6701,(X),50.6,0.3,49.4,0.3,NaN
3221,0500000US72149,"Villalba Municipio, Puerto Rico",21778,*****,10510,*****,11268,*****,93.3,*****,...,0.2,(X),(X),17649,(X),47.5,0.1,52.5,0.1,NaN
3222,0500000US72151,"Yabucoa Municipio, Puerto Rico",29868,*****,14381,*****,15487,*****,92.9,*****,...,0.2,(X),(X),24903,(X),47.3,0.2,52.7,0.2,NaN


Cleaning and dropping of columns from income and population datasets to keep only the needed ones.

In [40]:


# --- Clean County Population Data ---
df_population_clean = df_county_population_2023[[
    'Geography',
    'Geographic Area Name',
    'Estimate!!SEX AND AGE!!Total population'
]].copy()

df_population_clean.rename(columns={
    'Geography': 'GEO_ID',
    'Geographic Area Name': 'County_State',
    'Estimate!!SEX AND AGE!!Total population': 'Population_Total'
}, inplace=True)

# --- Clean Median Income Data ---
df_income_clean = df_median_income_by_county_2023[[
    'Geography',
    'Geographic Area Name',
    'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households'
]].copy()

df_income_clean.rename(columns={
    'Geography': 'GEO_ID',
    'Geographic Area Name': 'County_State',
    'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households': 'Median_Income'
}, inplace=True)

# --- Convert Population and Income columns to numeric safely ---
df_population_clean['Population_Total'] = pd.to_numeric(
    df_population_clean['Population_Total'].str.replace(',', ''), errors='coerce'
)
df_income_clean['Median_Income'] = pd.to_numeric(
    df_income_clean['Median_Income'].str.replace(',', ''), errors='coerce'
)

# --- Check cleaned results ---
print("✅ Population data (cleaned):")
display(df_population_clean.head())

print("✅ Income data (cleaned):")
display(df_income_clean.head())



✅ Population data (cleaned):


,GEO_ID,County_State,Population_Total
0,0100000US,United States,332387540
1,010C243US,United States -- Rural,66686736
2,0500000US01001,"Autauga County, Alabama",59285
3,0500000US01003,"Baldwin County, Alabama",239945
4,0500000US01005,"Barbour County, Alabama",24757


✅ Income data (cleaned):


,GEO_ID,County_State,Median_Income
0,0100000US,United States,127482865
1,010C243US,United States -- Rural,25251727
2,0500000US01001,"Autauga County, Alabama",22523
3,0500000US01003,"Baldwin County, Alabama",94642
4,0500000US01005,"Barbour County, Alabama",9080


In [41]:
print(df_county_population_2023.columns.tolist()[:20])  # first 20 column names
print(df_median_income_by_county_2023.columns.tolist()[:20])


['Geography', 'Geographic Area Name', 'Estimate!!SEX AND AGE!!Total population', 'Margin of Error!!SEX AND AGE!!Total population', 'Estimate!!SEX AND AGE!!Total population!!Male', 'Margin of Error!!SEX AND AGE!!Total population!!Male', 'Estimate!!SEX AND AGE!!Total population!!Female', 'Margin of Error!!SEX AND AGE!!Total population!!Female', 'Estimate!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females)', 'Margin of Error!!SEX AND AGE!!Total population!!Sex ratio (males per 100 females)', 'Estimate!!SEX AND AGE!!Total population!!Under 5 years', 'Margin of Error!!SEX AND AGE!!Total population!!Under 5 years', 'Estimate!!SEX AND AGE!!Total population!!5 to 9 years', 'Margin of Error!!SEX AND AGE!!Total population!!5 to 9 years', 'Estimate!!SEX AND AGE!!Total population!!10 to 14 years', 'Margin of Error!!SEX AND AGE!!Total population!!10 to 14 years', 'Estimate!!SEX AND AGE!!Total population!!15 to 19 years', 'Margin of Error!!SEX AND AGE!!Total population!!15 to 19 years'

Cleaning of Target Data 

In [43]:
import pandas as pd
import re

# 1) Load
df_target = pd.read_csv("Target datasets.csv", encoding="latin1")

# 2) Inspect columns (quick sanity)
print(df_target.columns.tolist()[:20])

# 3) Build a robust selector (handles missing 'Address.State')
keep_cols = [
    'Address.AddressLine1',
    'Address.City',
    'Address.County',
    'Address.FormattedAddress',
    'Address.Latitude',
    'Address.Longitude'
]
keep_cols = [c for c in keep_cols if c in df_target.columns]  # keep only those that exist

df_target_clean = df_target[keep_cols].copy()

# 4) Rename the ones we have
rename_map = {
    'Address.AddressLine1': 'Address',
    'Address.City': 'City',
    'Address.County': 'County',
    'Address.FormattedAddress': 'FormattedAddress',
    'Address.Latitude': 'Latitude',
    'Address.Longitude': 'Longitude'
}
df_target_clean.rename(columns={k:v for k,v in rename_map.items() if k in df_target_clean.columns},
                      inplace=True)

# 5) Clean County text
if 'County' in df_target_clean.columns:
    df_target_clean['County'] = df_target_clean['County'].astype(str)\
        .str.replace(' County', '', regex=False).str.strip()

# 6) Derive State (only if missing)
if 'State' not in df_target_clean.columns:
    # try to extract 2-letter state code from the formatted address (e.g., "Austin, TX 78704")
    if 'FormattedAddress' in df_target_clean.columns:
        df_target_clean['State'] = df_target_clean['FormattedAddress']\
            .str.extract(r',\s*([A-Z]{2})\s+\d{5}(?:-\d{4})?$')[0]
    else:
        df_target_clean['State'] = pd.NA  # fallback

# 7) Basic cleaning
df_target_clean.drop_duplicates(inplace=True)
df_target_clean.dropna(subset=['Latitude','Longitude'], inplace=True)

# Optional: standardize case
for col in ['City','County','State']:
    if col in df_target_clean.columns:
        df_target_clean[col] = df_target_clean[col].astype(str).str.strip()

print("✅ Cleaned Target data:")
display(df_target_clean.head())
print(df_target_clean.info())


['X.locale', 'Address.AddressLine1', 'Address.AddressLine2', 'Address.City', 'Address.CountryName', 'Address.County', 'Address.FormattedAddress', 'Address.IntersectionDescription', 'Address.Latitude', 'Address.Longitude', 'Address.PostalCode', 'Address.Subdivision', 'AlternateIdentifier.ID', 'ID', 'IsDaylightSavingsTimeRecognized', 'LocationMilestones.LastRemodelDate', 'LocationMilestones.OpenDate', 'Market', 'Name', 'OperatingHours..timeFormat']
✅ Cleaned Target data:


,Address,City,County,FormattedAddress,Latitude,Longitude,State
0,1901 13th Ave E,Tuscaloosa,Tuscaloosa,"1901 13th Ave E, Tuscaloosa, AL 35404-4785",33.193104,-87.518672,AL
1,2929 Watson Blvd,Warner Robins,Houston,"2929 Watson Blvd, Warner Robins, GA 31093-9600",32.616594,-83.686391,GA
2,916 Loganville Hwy,Bethlehem,Barrow,"916 Loganville Hwy, Bethlehem, GA 30620-2145",33.942788,-83.752011,GA
3,5865 Spout Springs Rd,Flowery Branch,Hall,"5865 Spout Springs Rd, Flowery Branch, GA 3054...",34.178722,-83.911284,GA
4,2750 Carl T Jones Dr SE,Huntsville,Madison,"2750 Carl T Jones Dr SE, Huntsville, AL 35802-...",34.673498,-86.541632,AL


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1780 entries, 0 to 1779
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address           1780 non-null   object 
 1   City              1780 non-null   object 
 2   County            1780 non-null   object 
 3   FormattedAddress  1780 non-null   object 
 4   Latitude          1780 non-null   float64
 5   Longitude         1780 non-null   float64
 6   State             1780 non-null   object 
dtypes: float64(2), object(5)
memory usage: 97.5+ KB
None


In [45]:
# Combine population and income datasets
df_demographics = pd.merge(
    df_population_clean,
    df_income_clean[['County_State', 'Median_Income']],
    on='County_State',
    how='left'
)

# Clean up the County_State field for consistent merging
df_demographics['County_State'] = (
    df_demographics['County_State']
    .str.replace(' County', '', regex=False)
    .str.strip()
)

print("✅ Combined Demographics Data:")
display(df_demographics.head())


✅ Combined Demographics Data:


,GEO_ID,County_State,Population_Total,Median_Income
0,0100000US,United States,332387540,127482865
1,010C243US,United States -- Rural,66686736,25251727
2,0500000US01001,"Autauga, Alabama",59285,22523
3,0500000US01003,"Baldwin, Alabama",239945,94642
4,0500000US01005,"Barbour, Alabama",24757,9080


In [53]:
import pandas as pd

# 1) Map state abbreviations -> full names
us_state_map = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado',
    'CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho',
    'IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana',
    'ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota','MS':'Mississippi',
    'MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey',
    'NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
    'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
    'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming','DC':'District of Columbia','PR':'Puerto Rico'
}

# 2) Target: county + full state, normalized
df_target_clean['State_Full'] = df_target_clean['State'].map(us_state_map)
df_target_clean['county_state_key'] = (
    df_target_clean['County'].str.strip().str.lower() + '|' +
    df_target_clean['State_Full'].str.strip().str.lower()
)

# 3) Census: split County_State -> County + State, then key
# (Assumes you already have df_demographics from earlier)
demo = df_demographics.copy()
demo[['County_only','State_full']] = demo['County_State'].str.split(', ', n=1, expand=True)
demo['county_state_key'] = (
    demo['County_only'].str.strip().str.lower() + '|' +
    demo['State_full'].str.strip().str.lower()
)

# 4) Quick match-rate check
match_rate = df_target_clean['county_state_key'].isin(demo['county_state_key']).mean()
print(f"Estimated match rate: {match_rate:.1%}")

# 5) Ready-to-merge tables
print(df_target_clean[['County','State','State_Full','county_state_key']].head())
print(demo[['County_only','State_full','Population_Total','Median_Income','county_state_key']].head())


Estimated match rate: 98.3%
       County State State_Full    county_state_key
0  Tuscaloosa    AL    Alabama  tuscaloosa|alabama
1     Houston    GA    Georgia     houston|georgia
2      Barrow    GA    Georgia      barrow|georgia
3        Hall    GA    Georgia        hall|georgia
4     Madison    AL    Alabama     madison|alabama
              County_only State_full  Population_Total  Median_Income  \
0           United States       None         332387540      127482865   
1  United States -- Rural       None          66686736       25251727   
2                 Autauga    Alabama             59285          22523   
3                 Baldwin    Alabama            239945          94642   
4                 Barbour    Alabama             24757           9080   

  county_state_key  
0              NaN  
1              NaN  
2  autauga|alabama  
3  baldwin|alabama  
4  barbour|alabama  


In [59]:
# 1) Recompute store counts PER COUNTY using only the key (avoid bringing County/State to prevent suffixes)
stores_per_county = (
    df_merged.groupby('county_state_key')
             .size()
             .rename('Store_Count')
             .reset_index()
)

# 2) Prepare a clean demographics table with canonical names
demo_base = (
    demo_valid[['county_state_key','County_only','State_full','Population_Total','Median_Income']]
    .rename(columns={'County_only':'County', 'State_full':'State'})
    .copy()
)

# 3) Merge (no duplicate columns now)
county_view = pd.merge(
    demo_base,
    stores_per_county,           # only key + Store_Count
    on='county_state_key',
    how='left'
)

# 4) Clean types and compute density
county_view['Store_Count'] = county_view['Store_Count'].fillna(0).astype(int)
county_view['Population_Total'] = pd.to_numeric(county_view['Population_Total'], errors='coerce')
county_view['Median_Income']    = pd.to_numeric(county_view['Median_Income'], errors='coerce')
county_view['Stores_per_100k']  = (county_view['Store_Count'] / (county_view['Population_Total']/100000)).round(3)

print("County-level summary (first 10):")
display(county_view.head(10)[['County','State','Population_Total','Median_Income','Store_Count','Stores_per_100k']])


County-level summary (first 10):


,County,State,Population_Total,Median_Income,Store_Count,Stores_per_100k
0,Autauga,Alabama,59285,22523,0,0.000
1,Baldwin,Alabama,239945,94642,2,0.834
2,Barbour,Alabama,24757,9080,0,0.000
3,Bibb,Alabama,22152,7571,0,0.000
4,Blount,Alabama,59292,21977,0,0.000
5,Bullock,Alabama,10157,3453,0,0.000
6,Butler,Alabama,18807,7262,0,0.000
7,Calhoun,Alabama,116141,45125,0,0.000
8,Chambers,Alabama,34450,13419,0,0.000
9,Cherokee,Alabama,25224,10477,0,0.000


In [60]:
opportunity = (
    county_view
      .query("Store_Count == 0 and Population_Total >= 150_000 and Median_Income >= 60000")
      .sort_values(['Population_Total','Median_Income'], ascending=False)
      .head(25)
      .reset_index(drop=True)
)

display(opportunity[['County','State','Population_Total','Median_Income','Store_Count']])


,County,State,Population_Total,Median_Income,Store_Count
0,Fulton,Georgia,1068507,457791,0
1,Capitol Planning Region,Connecticut,969029,389086,0
2,Gwinnett,Georgia,966972,322852,0
3,Cobb,Georgia,769152,293410,0
4,DeKalb,Georgia,762105,284681,0
5,Jefferson,Alabama,669744,268911,0
6,Western Connecticut Planning Region,Connecticut,621232,232279,0
7,South Central Connecticut Planning Region,Connecticut,566803,227739,0
8,Naugatuck Valley Planning Region,Connecticut,452303,178340,0
9,Pinal,Arizona,449219,157193,0


In [61]:
print(sorted(county_view.columns))


['County', 'Median_Income', 'Population_Total', 'State', 'Store_Count', 'Stores_per_100k', 'county_state_key']


In [62]:
# Keep only valid county rows in demographics (drop 'United States' etc.)
demo_valid = demo[demo['State_full'].notna()].copy()

# Store-level merge: link each store with its county demographics
df_merged = pd.merge(
    df_target_clean,
    demo_valid[['county_state_key', 'County_only', 'State_full', 'Population_Total', 'Median_Income']],
    on='county_state_key',
    how='left'
)

print("Store-level merged sample:")
display(df_merged.head())

# Merge coverage check
total = len(df_target_clean)
matched = df_merged['Population_Total'].notna().sum()
print(f"Matched stores: {matched}/{total}  ({matched/total:.1%})")


Store-level merged sample:


,Address,City,County,FormattedAddress,Latitude,Longitude,State,State_Full,county_state_key,County_only,State_full,Population_Total,Median_Income
0,1901 13th Ave E,Tuscaloosa,Tuscaloosa,"1901 13th Ave E, Tuscaloosa, AL 35404-4785",33.193104,-87.518672,AL,Alabama,tuscaloosa|alabama,Tuscaloosa,Alabama,234036.0,87289.0
1,2929 Watson Blvd,Warner Robins,Houston,"2929 Watson Blvd, Warner Robins, GA 31093-9600",32.616594,-83.686391,GA,Georgia,houston|georgia,Houston,Georgia,166902.0,62158.0
2,916 Loganville Hwy,Bethlehem,Barrow,"916 Loganville Hwy, Bethlehem, GA 30620-2145",33.942788,-83.752011,GA,Georgia,barrow|georgia,Barrow,Georgia,86982.0,29770.0
3,5865 Spout Springs Rd,Flowery Branch,Hall,"5865 Spout Springs Rd, Flowery Branch, GA 3054...",34.178722,-83.911284,GA,Georgia,hall|georgia,Hall,Georgia,208395.0,71263.0
4,2750 Carl T Jones Dr SE,Huntsville,Madison,"2750 Carl T Jones Dr SE, Huntsville, AL 35802-...",34.673498,-86.541632,AL,Alabama,madison|alabama,Madison,Alabama,397135.0,162469.0


Matched stores: 1750/1780  (98.3%)


In [63]:
# 1) Store counts per county
stores_per_county = (
    df_merged.groupby('county_state_key')
             .size()
             .rename('Store_Count')
             .reset_index()
)

# 2) Prepare demographics base table
demo_base = (
    demo_valid[['county_state_key','County_only','State_full','Population_Total','Median_Income']]
    .rename(columns={'County_only':'County', 'State_full':'State'})
    .copy()
)

# 3) Merge to create county-level view
county_view = pd.merge(
    demo_base,
    stores_per_county,
    on='county_state_key',
    how='left'
)

# 4) Clean and compute store density
county_view['Store_Count'] = county_view['Store_Count'].fillna(0).astype(int)
county_view['Population_Total'] = pd.to_numeric(county_view['Population_Total'], errors='coerce')
county_view['Median_Income'] = pd.to_numeric(county_view['Median_Income'], errors='coerce')
county_view['Stores_per_100k'] = (county_view['Store_Count'] / (county_view['Population_Total']/100000)).round(3)

display(county_view.head(10))


,county_state_key,County,State,Population_Total,Median_Income,Store_Count,Stores_per_100k
0,autauga|alabama,Autauga,Alabama,59285,22523,0,0.000
1,baldwin|alabama,Baldwin,Alabama,239945,94642,2,0.834
2,barbour|alabama,Barbour,Alabama,24757,9080,0,0.000
3,bibb|alabama,Bibb,Alabama,22152,7571,0,0.000
4,blount|alabama,Blount,Alabama,59292,21977,0,0.000
5,bullock|alabama,Bullock,Alabama,10157,3453,0,0.000
6,butler|alabama,Butler,Alabama,18807,7262,0,0.000
7,calhoun|alabama,Calhoun,Alabama,116141,45125,0,0.000
8,chambers|alabama,Chambers,Alabama,34450,13419,0,0.000
9,cherokee|alabama,Cherokee,Alabama,25224,10477,0,0.000


In [ ]:


# --- Clean County Population Data ---
df_population_clean = df_county_population_2023[[
    'Geography',
    'Geographic Area Name',
    'Estimate!!SEX AND AGE!!Total population'
]].copy()

df_population_clean.rename(columns={
    'Geography': 'GEO_ID',
    'Geographic Area Name': 'County_State',
    'Estimate!!SEX AND AGE!!Total population': 'Population_Total'
}, inplace=True)

# --- Clean Median Income Data ---
df_income_clean = df_median_income_by_county_2023[[
    'Geography',
    'Geographic Area Name',
    'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households'
]].copy()

df_income_clean.rename(columns={
    'Geography': 'GEO_ID',
    'Geographic Area Name': 'County_State',
    'Estimate!!Number!!HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER!!Households': 'Median_Income'
}, inplace=True)

# --- Convert Population and Income columns to numeric safely ---
df_population_clean['Population_Total'] = pd.to_numeric(
    df_population_clean['Population_Total'].str.replace(',', ''), errors='coerce'
)
df_income_clean['Median_Income'] = pd.to_numeric(
    df_income_clean['Median_Income'].str.replace(',', ''), errors='coerce'
)

# --- Check cleaned results ---
print("✅ Population data (cleaned):")
display(df_population_clean.head())

print("✅ Income data (cleaned):")
display(df_income_clean.head())



✅ Population data (cleaned):


,GEO_ID,County_State,Population_Total
0,0100000US,United States,332387540
1,010C243US,United States -- Rural,66686736
2,0500000US01001,"Autauga County, Alabama",59285
3,0500000US01003,"Baldwin County, Alabama",239945
4,0500000US01005,"Barbour County, Alabama",24757


✅ Income data (cleaned):


,GEO_ID,County_State,Median_Income
0,0100000US,United States,127482865
1,010C243US,United States -- Rural,25251727
2,0500000US01001,"Autauga County, Alabama",22523
3,0500000US01003,"Baldwin County, Alabama",94642
4,0500000US01005,"Barbour County, Alabama",9080


In [64]:
opportunity = (
    county_view
      .query("Store_Count == 0 and Population_Total >= 150_000 and Median_Income >= 60000")
      .sort_values(['Population_Total','Median_Income'], ascending=False)
      .head(25)
      .reset_index(drop=True)
)

print("Top opportunity counties (no stores, high pop & income):")
display(opportunity[['County','State','Population_Total','Median_Income','Store_Count']])


Top opportunity counties (no stores, high pop & income):


,County,State,Population_Total,Median_Income,Store_Count
0,Fulton,Georgia,1068507,457791,0
1,Capitol Planning Region,Connecticut,969029,389086,0
2,Gwinnett,Georgia,966972,322852,0
3,Cobb,Georgia,769152,293410,0
4,DeKalb,Georgia,762105,284681,0
5,Jefferson,Alabama,669744,268911,0
6,Western Connecticut Planning Region,Connecticut,621232,232279,0
7,South Central Connecticut Planning Region,Connecticut,566803,227739,0
8,Naugatuck Valley Planning Region,Connecticut,452303,178340,0
9,Pinal,Arizona,449219,157193,0


Exporting files for store, county and opportunity 

In [65]:
df_store_export = df_merged[['Address','City','County','State','Latitude','Longitude','Population_Total','Median_Income']]
df_store_export.to_csv("target_store_level.csv", index=False)


In [66]:
county_export = county_view[['County','State','Population_Total','Median_Income','Store_Count']]
county_export.to_csv("county_level_summary.csv", index=False)


In [67]:
opportunity[['County','State','Population_Total','Median_Income','Store_Count']].to_csv("target_expansion_opportunities.csv", index=False)
